# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
import tempfile
import subprocess
from pydub import AudioSegment
import time
import anthropic

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

openai = OpenAI()
claude = anthropic.Anthropic()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant. "
system_message += "Give short, courteous answers. You can check ticket price, ticket availability, and reserve tickets for users. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [5]:
ticket_availability = {"london": "20", "paris": "90", "tokyo": "100", "berlin": "2"}

In [6]:
# Let's start by making a useful function
def get_ticket_availability(destination_city):
    print(f"Tool get_ticket_availability called for {destination_city}")
    available = destination_city.lower()
    return ticket_availability.get(available, "Unknown")

In [7]:
def get_ticket_price_availability(destination_city):
    print(f"Tool get_ticket_price_availability called for {destination_city}")
    available = destination_city.lower()
    price = destination_city.lower()
    return ticket_price.get(price, "Unknown"), ticket_availability.get(available, "Unknown")

In [8]:
def book_ticket(destination_city,price,availability,no_of_tickets):
    status=""
    if availability == 0:
        status="Cannot book a ticket, no seat available\n"
    elif (int(availability)-int(no_of_tickets)) < 0:
        status="Cannot book a ticket, no seat available\n"
    else:
        print(f"Tool book_function called for {destination_city}")
        f = open("C:/Users/aruna/Desktop/book_status.txt", "a")
        f.write(f"{no_of_tickets} ticket/s to {destination_city} booked for {price} x {no_of_tickets}, currently available - {int(availability)-int(no_of_tickets)}")
        f.write("\n")
        f.close()
        ticket_availability[destination_city.lower()]=str(int(availability)-int(no_of_tickets))
        
        status="Ticket reservation is a success\n"
    return status

In [9]:
book_function = {
    "name": "book_ticket",
    "description": "Book the ticket based on the ticket price and/ or availability as requested by the user. For example, when a \
     user asks to purchase one or more tickets to Tokyo after getting to know the ticket price and/or the availability, then \
     proceed with this tool call. Else, request the user to either ask for ticket price or availability first. Please help the customer in booking the ticket/s if tickets are available. But before each time\
     you book, ask confirmation from the user before proceeding with booking.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "string",
                "description": "The price of the ticket to the city",
            },
            "availability": {
                "type": "string",
                "description": "ticket availability to the city the customer wants to travel to",
            },
            "no_of_tickets": {
                "type": "string",
                "description": "the number of tickets the customer wants to purchase/ book to the destination",
            }
        },
        "required": ["destination_city","price","availability","no_of_tickets"],
        "additionalProperties": False
    }
}

In [10]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a one_way ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [11]:
availability_function = {
    "name": "get_ticket_availability",
    "description": "Get the availability of a one-way ticket to the destination city. Call this whenever you need to know the ticket availability, for example when a customer asks 'What is the ticket availability to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [12]:
ticket_price_availability_function = {
    "name": "get_ticket_price_availability",
    "description": "Get the price or availability of a one-way ticket to the destination city. Call this whenever you need to know the ticket price and availability, for example when a customer asks 'What is the ticket availability and price to this city'\
    or 'what is the price and ticket for the city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [13]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": availability_function},
         {"type": "function", "function": ticket_price_availability_function},
         {"type": "function", "function": book_function}]

In [14]:
def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",  # Also, try replacing with onyx
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)


In [15]:
# We have to write that function handle_tool_call:

def handle_tool_call1(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    name = json.dumps(tool_call.function.name)
    city = arguments.get('destination_city')
    no = arguments.get('no_of_tickets')
    
    if name.replace('"','') == "get_ticket_price":
        price = get_ticket_price(city)
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
        }
    elif name.replace('"','') == "book_ticket":
        price = get_ticket_price(city)
        availability = get_ticket_availability(city)
        booked=book_ticket(city,price,availability,no)
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "number of tickets":no, "booking_status": booked}),
        "tool_call_id": tool_call.id
        }
    else :
        availability = get_ticket_availability(city)
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"availability": availability},),
        "tool_call_id": tool_call.id
        }
        
    return response, city

In [16]:
def chat_open_ai(history):
    messages = [{"role": "system", "content": system_message}] + history 
    response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)
    if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            print(message)
            tool_call = message.tool_calls[0]
            arguments = json.loads(tool_call.function.arguments)
            name = json.dumps(tool_call.function.name)
            city = arguments.get('destination_city')
    
            if name.replace('"','') == "book_ticket":
                response, city = handle_tool_call1(message)
                messages.append(message)
                messages.append(response)
                # image = artist(city)
                response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)
            elif name.replace('"','') == "get_ticket_price_availability":
                price = get_ticket_price(city)
                availability = get_ticket_availability(city)
                response = {
                                "role": "tool",
                                "content": json.dumps({"destination_city": city,"price": price,"availability": availability}),
                                "tool_call_id": tool_call.id
                            }
                messages.append(message)
                messages.append(response)
                print(messages)
                response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)                    
            else:    
                response, city = handle_tool_call1(message)
                messages.append(message)
                messages.append(response)
                # image = artist(city)
                response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)
            
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    # return history, image
    return history

In [17]:
tools_claude=[
                {
                    "name": "get_ticket_price_availability",
                    "description": "Get the availability of a one-way ticket to the destination city or the price. Call this whenever you need to know the ticket price or availability or both, for example, when a customer asks 'What is the ticket availability and/ or price to this city'",
                    "input_schema": {
                                    "type": "object",
                                    "properties": {
                                    "destination_city": {
                                                            "type": "string",
                                                            "description": "The city that the customer wants to travel to",
                                                        },
                                                  },
                                    "required": ["destination_city"]
                                    ,"additionalProperties": False
                                    }
                }
            ]
tool_choice = [{"type": "tool", "name": "get_ticket_price_availability"}]

In [18]:
def chat_claude(history):
    for element in history:
        del element["metadata"]
        del element["options"]

    messages = history
    response = claude.messages.create(
        model=claude_model,
        system=system_message,
        messages=messages,
        max_tokens=500,
        tools=tools_claude
    )
    print(response.content[0])
    if response.stop_reason=="tool_use":        
        if "text=" in str(response.content[0]):
        # if response.content[0].text is None:
                tool_name = response.content[1].name
                tool_input = response.content[1].input
                tool_id = response.content[1].id
                tool_use=response.content[1]
        else:
                tool_name = response.content[0].name
                tool_input = response.content[0].input
                tool_id = response.content[0].id
                tool_use=response.content[0]
        
     
        city = tool_input.get('destination_city')    
        if tool_name == "get_ticket_price_availability":
            price = get_ticket_price(city)
            availability = get_ticket_availability(city)
            result_dict = {
                'destination_city': city,
                'price': price,
                'availability': availability
            }
            messages += [{"role": "user","content": json.dumps(result_dict)}]
        response = claude.messages.create(
                model=claude_model,
                system=system_message,
                messages=messages,
                max_tokens=500,
                # tools=tools_claude
            )    
        history.pop(len(history)-1)
        print(history)
    reply = response.content[0].text
    history += [{"role":"assistant", "content":reply}]
    # Comment out or delete the next line if you'd rather skip Audio for now..
    # talker(reply)
    
    # return history, image
    return history

In [19]:
def chat1(history, Model):
    # + [{"role": "user", "content": message}]
    if Model=="Open AI":
        history = chat_open_ai(history)
    else:
        history = chat_claude(history)
    return history

In [20]:
def listen2(history):
    import speech_recognition as sr

    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = r.listen(source, phrase_time_limit=30)
        text=""
        try:
            text = r.recognize_google(audio)
            print("You said:", text)
        except sr.UnknownValueError:
            print("Could not understand audio.")

        history += [{"role":"user", "content":text}]        
        return "", history

In [21]:
# import speech_recognition as sr
# from pydub import AudioSegment
# import simpleaudio as sa

# def listener():
#     recognizer = sr.Recognizer()
    
#     with sr.Microphone() as source:
#         print("Listening... Speak now!")
#         recognizer.adjust_for_ambient_noise(source)  # Adjust for background noise
#         audio = recognizer.listen(source, phrase_time_limit=30)
    
#     try:
#         print("Processing speech...")
#         text = recognizer.recognize_google(audio)  # Use Google Speech-to-Text
#         print(f"You said: {text}")
#         return text
#     except sr.UnknownValueError:
#         print("Sorry, I could not understand what you said.")
#         return None
#     except sr.RequestError:
#         print("Could not request results, please check your internet connection.")
#         return None

# # Example usage:
# # text = listener()  # Listen for speech
# # if text:
# #     print(f"You just said: {text}") 

In [22]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=300, type="messages")
        # image_output = gr.Image(height=500)
    with gr.Row():
        Model = gr.Dropdown(["Open AI","Claude"],
                              # value=["Open AI","Claude"],
                              multiselect=False,
                              label="Model",
                              interactive=True)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        speak = gr.Button("click for voice search") 
    with gr.Row():
        clear = gr.Button("Clear")

    def listen(history):
        message, history=listen2(history)
        return message, history

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        # chat, inputs=chatbot, outputs=[chatbot, image_output]
        chat1, inputs=[chatbot, Model], outputs=[chatbot]
    )
    speak.click(listen, inputs=[chatbot], outputs=[entry, chatbot]).then(
        chat1, inputs=[chatbot, Model], outputs=[chatbot]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [23]:
print(ticket_availability)

{'london': '20', 'paris': '90', 'tokyo': '100', 'berlin': '2'}
Speak now...
You said: price of tickets to Tokyo
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ddXzm2cPBJ9SOsV8qI4L2FcB', function=Function(arguments='{"destination_city":"Tokyo"}', name='get_ticket_price'), type='function')])
Tool get_ticket_price called for Tokyo
Speak now...
You said: what is the price of two tickets to London
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lSNZCwaUdckvk3V0eTBlotRN', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')])
Tool get_ticket_price called for London
Speak now...
You said: can you please reserve two tickets for me to London
ChatCompletionMessage(content='First, I need to check the availabilit